In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[ WARN] [1710108074.901119891]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 195.410000 according to authority /pose_integrator
[ WARN] [1710108075.412574347]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 195.646000 according to authority /pose_integrator
[ WARN] [1710108076.979045929]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 196.061000 according to authority /pose_integrator
[ WARN] [1710108078.923022172]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 196.956000 according to authority /pose_integrator
[ WARN] [1710108078.923082929]: TF_REPEATED_DATA ignoring data with redundant timestamp fo

In [ ]:
arm = moveit_commander.MoveGroupCommander('arm')

In [ ]:
head.set_joint_values([0.0,-0.65])
arm.set_named_target('go')
arm.go()

In [ ]:

rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
omni_base.move_d_to(target_distance=0.78,target_link='apple')

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
for i in range(len(res.names)):
    if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
        bbM=res.pt_min.data[2*i:2*i+2]
        bbm=res.pt_max.data[2*i:2*i+2]
        bbx=[bbM,bbm]
print(bbx)
plt.imshow(debug_image)

# [(297, 191), (231, 226)]  at pkcp pose #Real Tak
### [(323, 266), (280, 215)]  at pkcp pose #Gazebo


In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eY=(bbx[1][1]-239)/239
    eX=(bbx[0][0]-312)/312
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
bbx

In [ ]:
eY=(bbx[1][1]-239)/239
eX=(bbx[0][0]-312)/312
print(f'error relative to gripper cam ex={eX} ey={eY}')


In [ ]:
#omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 


In [ ]:
clear_octo_client()

av=arm.get_current_joint_values()
print (av,'av')
av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
plt.imshow(hand_cam.get_image())

In [ ]:
image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
plt.imshow(debug_image)

In [ ]:
gripper.open()

In [ ]:
gripper.close(force=0.06)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
    tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])




In [ ]:
def detect_object_yolo(object_name,res):
    # find object_name in the response message from object_classification service (Yolo)
    objs=[]
    for i,name in enumerate(res.names):
        objs.append(name.data[4:])
        if name.data[4:]==object_name:return res.poses[i]
    if object_name=='all':
        
        return objs
    return []

In [ ]:
#img_msg  = bridge.cv2_to_imgmsg(rgbd.get_image())  #REAL
img_msg  = bridge.cv2_to_imgmsg(cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR ))#GAZEB
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   
if len (objects)!=0:
    for i in range(len(res.poses)):
        tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(res.names[i].data[4:])
        rospy.sleep(0.3)
        pose , _=tf_man.getTF(res.names[i].data[4:])
        new_row = {'x': pose[0], 'y': pose[1], 'z': pose[2], 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
objs

In [ ]:
objs[['x','y','z']].values

In [ ]:
#get closest object
rob_pos,_=tf_man.getTF('base_link')
np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))
objs.iloc[np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))]['obj_name']

In [ ]:
objs[objs['obj_name']==objs.obj_name.values[0]].x.values[0]

# PLANE 1
#### 7.883384	5.948245	0.701945,
#### 8.214565	5.962619	0.700034,
#### 8.213697	6.107727	0.709487,
####  7.893623	6.082935	0.693285

# PLANE 2
#### 7.88076	6.086619	0.416096,
#### 8.214128	5.9663	0.416672

# Plane 3
#### 8.20238	5.963262	0.117541

In [ ]:
head.set_named_target('neutral')

av=arm.get_current_joint_values()
av[0]=0.5
av[1]=-0.5
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:


av=arm.get_current_joint_values()
av[0]=0.05

arm.go(av)

In [ ]:
head.set_joint_values([-np.pi/2 , -0.8])

av=arm.get_current_joint_values()
av[0]=0.0
av[1]=-0.5
arm.go(av)

In [1]:
import pandas as pd
objs = pd.read_csv ('/home/roboworks/Documents/objs.csv')
objs=objs.drop(columns='Unnamed: 0')

/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
objs

,x,y,z,obj_name,shelves,pickup,category
0,6.175408,4.640078,0.421324,bowl,False,True,kitchen
1,6.363730,4.576172,0.441157,a_pitcher_base,False,True,other
2,6.290324,4.533250,0.289092,b_colored_wood_blocks,False,True,other
3,5.938471,4.671816,0.499365,a_colored_wood_blocks,False,True,other
4,6.062599,4.534405,0.289561,b_colored_wood_blocks,False,True,other
5,5.975489,4.534365,0.290384,b_colored_wood_blocks,False,True,other
6,6.382711,4.541311,0.283372,b_colored_wood_blocks,False,True,other
7,8.349535,5.904183,0.748459,a_pitcher_base,True,False,other
8,8.367075,5.875787,0.722083,a_colored_wood_blocks,True,False,other
9,8.277662,6.062730,0.687708,master_chef_can,True,False,food


In [26]:
from sklearn.cluster import KMeans
shelf_objs=objs[objs['shelves']==True ]
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]


In [33]:
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]
sh_o_mdl=shelf_objs[(shelf_objs['z']>0.2 )& (shelf_objs['z']<0.5) ]
sh_o_top=shelf_objs[(shelf_objs['z']>0.5 )& (shelf_objs['z']<1.5) ]
sh_o_top

,x,y,z,obj_name,shelves,pickup,category
7,8.349535,5.904183,0.748459,a_pitcher_base,True,False,other
8,8.367075,5.875787,0.722083,a_colored_wood_blocks,True,False,other
9,8.277662,6.062730,0.687708,master_chef_can,True,False,food
10,8.246050,5.958805,0.713797,a_pitcher_base,True,False,other
11,7.995178,6.150285,0.830315,b_cups,True,False,other
12,8.008332,6.063715,0.742864,a_pitcher_base,True,False,other
13,8.046444,6.146267,0.701318,a_pitcher_base,True,False,other
14,8.282028,6.070728,0.667506,a_pitcher_base,True,False,other
17,8.174064,6.047288,0.659112,plate,True,False,kitchen


In [3]:
kitchen =['bowl','spatula','spoon', 'bowl']
tools=['extra_large_clamp','large_clamp','small_clamp','medium_clamp','adjustable_wrench',
'flat_screwdriver','phillips_screwdriver','wood_block']
balls= ['softball','tennis_ball']
fruits= ['apple','banana', 'lemon','pear']
food =['chips_can'
,'mustard_bottle'
,'potted_meat_can'
,'tomato_soup_can'
,'tuna_fish_can']

def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats   


In [4]:
def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
    
    

In [5]:
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats

In [6]:
objs

,x,y,z,obj_name,category
0,5.980992,4.677664,0.506293,a_colored_wood_blocks,other
1,6.219399,4.659809,0.421303,bowl,kitchen
2,5.928290,4.551338,0.290167,b_colored_wood_blocks,other
3,6.390869,4.590297,0.431736,a_pitcher_base,other
4,8.230576,6.101967,0.704379,master_chef_can,other
5,8.221585,5.984989,0.944968,nine_hole_peg_test,other
6,8.032595,5.981796,0.418046,h_cups,other
7,7.947621,6.034316,0.421347,softball,balls
8,8.154552,5.987593,0.688931,tuna_fish_can,food
9,8.119175,6.098844,0.087389,large_clamp,tools


In [9]:
regions={'shelves':np.load('/home/roboworks/Documents/shelves_region.npy'),'pickup':np.load('/home/roboworks/Documents/pickup_region.npy')}
def is_inside(x,y):return ((area_box[1,1] > y) and (area_box[0,1] < y)) and ((area_box[1,0] > x) and (area_box[0,0] < x)) 
for name in regions:
    in_region=[]
    area_box=regions[name]
    print (area_box)
    for index, row in objs[['x','y']].iterrows():in_region.append(is_inside(row.x, row.y))
    objs[name]=pd.Series(in_region)
objs

[[ 7.85000012  4.50000007]
 [11.35000017  8.55000013]]
[[3.80000006 3.60000005]
 [7.80000012 8.40000013]]


,x,y,z,obj_name,category,shelves,pickup
0,5.980992,4.677664,0.506293,a_colored_wood_blocks,other,False,True
1,6.219399,4.659809,0.421303,bowl,kitchen,False,True
2,5.928290,4.551338,0.290167,b_colored_wood_blocks,other,False,True
3,6.390869,4.590297,0.431736,a_pitcher_base,other,False,True
4,8.230576,6.101967,0.704379,master_chef_can,other,True,False
5,8.221585,5.984989,0.944968,nine_hole_peg_test,other,True,False
6,8.032595,5.981796,0.418046,h_cups,other,True,False
7,7.947621,6.034316,0.421347,softball,balls,True,False
8,8.154552,5.987593,0.688931,tuna_fish_can,food,True,False
9,8.119175,6.098844,0.087389,large_clamp,tools,True,False


In [8]:
import numpy as np